In [2]:
import pandas as pd
import numpy as np
from skyfield.api import load, Topos, Star, utc
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

import os
import pathlib
import stars_utils

MODEL_DIR = 'models'
MODEL_FILENAME = 'mars_position_predictor_final.keras' 

In [2]:
# Load the JPL ephemeris data file
planets = load('de421.bsp')
ts = load.timescale()
mars = planets['mars']
earth = planets['earth']

# 2. Define Time Range (e.g., 50 years of data, 1970 to 2020)
start_date = datetime(1970, 1, 1)
end_date = datetime(2025, 1, 1)
time_step = timedelta(days=1)  # Data point every week

# Generate a list of dates
dates = []
current_date = start_date
while current_date <= end_date:
    dates.append(current_date)
    current_date += time_step


In [3]:

# Convert Python datetimes to Skyfield time objects
timezone_aware_dates = [d.replace(tzinfo=utc) for d in dates]
t = ts.utc(timezone_aware_dates)

# 3. Calculate Geocentric Position (Position as seen from Earth)
# astrometric_position is the calculation of a body's position in space
# as seen from a specific location (Earth).
astrometric = earth.at(t).observe(mars)

# Get Right Ascension and Declination
# These are your primary target variables for prediction!
ra, dec, distance = astrometric.radec()

# 4. Create the DataFrame (Your Dataset)
data = {
    'Time_UTC': dates,
    'Julian_Date': t.tdb,
    'RA_deg': ra.degrees,      # Right Ascension (Target Variable)
    'Dec_deg': dec.degrees     # Declination (Target Variable)
}

df = pd.DataFrame(data)

# 5. Save the Dataset
df.to_csv('mars_ephemeris_data.csv', index=False)
print("Dataset created successfully with", len(df), "data points.")

mars_sf_df = df.copy()

Dataset created successfully with 20090 data points.


In [5]:
mars_sf_df.shape

(20090, 9)

In [8]:
# get the astronomic xyz coordinates of mars from earth's pov
astrometric = earth.at(t).observe(mars)
pos_vector_au = astrometric.xyz.au #pos_vector_au is a 3xN NumPy array, where N is the number of time steps (t).

data = {
    'Time_UTC': dates,
    'Julian_Date': t.tdb,
    
    # Extract the X, Y, and Z components
    'X_au': pos_vector_au[0],  # X-coordinate
    'Y_au': pos_vector_au[1],  # Y-coordinate
    'Z_au': pos_vector_au[2]   # Z-coordinate
}

mars_sf_df = pd.DataFrame(data)

In [ ]:
JD_min = mars_sf_df.Julian_Date.min()
mars_sf_df["Time_Index"] = mars_sf_df.Julian_Date - JD_min
time_index = mars_sf_df["Time_Index"]

mars_sf_df["Time_Index_2"] = time_index ** 2
mars_sf_df["Time_Index_3"] = time_index ** 3

# Add features to the dataframe
PERIOD_YEAR = 365.25
PERIOD_MARS = 686.98
PERIOD_SYNODIC = 780.0
PERIOD_JUPITER = 4332.6 # mars cycles are influenced by jupiters movement. added this for fixing.

# Earth's Annual Cycle Features
mars_sf_df["Sin_Year"] = np.sin(2*np.pi*time_index/PERIOD_YEAR)
mars_sf_df["Cos_Year"] = np.cos(2*np.pi*time_index/PERIOD_YEAR)

# Mars's Orbital Cycle Features (NEW and CRUCIAL)
mars_sf_df['Sin_Mars'] = np.sin(2 * np.pi * time_index / PERIOD_MARS)
mars_sf_df['Cos_Mars'] = np.cos(2 * np.pi * time_index / PERIOD_MARS)

# Mars-Earth Synodic Cycle Features
mars_sf_df["Sin_Synodic"] = np.sin(2*np.pi*time_index/PERIOD_SYNODIC)
mars_sf_df["Cos_Synodic"] = np.cos(2*np.pi*time_index/PERIOD_SYNODIC)

# Jupiter's Annual Cycle Features
mars_sf_df['Sin_Jupiter'] = np.sin(2 * np.pi * time_index / PERIOD_JUPITER)
mars_sf_df['Cos_Jupiter'] = np.cos(2 * np.pi * time_index / PERIOD_JUPITER)

mars_sf_df['Sin_Year_Sin_Synodic'] = mars_sf_df['Sin_Year'] * mars_sf_df['Sin_Synodic']
mars_sf_df['Sin_Year_Cos_Synodic'] = mars_sf_df['Sin_Year'] * mars_sf_df['Cos_Synodic']
mars_sf_df['Cos_Year_Sin_Synodic'] = mars_sf_df['Cos_Year'] * mars_sf_df['Sin_Synodic']
mars_sf_df['Cos_Year_Cos_Synodic'] = mars_sf_df['Cos_Year'] * mars_sf_df['Cos_Synodic']

# Define features (x) and targets (y) for the model 
FEATURES = [
    'Time_Index', 'Time_Index_2', 'Time_Index_3', 
    'Sin_Year', 'Cos_Year',
    'Sin_Mars', 'Cos_Mars',
    'Sin_Synodic', 'Cos_Synodic',
]
TARGETS = ['X_au', 'Y_au', 'Z_au']


In [1]:
X = mars_sf_df[FEATURES]
y = mars_sf_df[TARGETS]

# 80/20 split of the data
split_point = int(len(mars_sf_df)*0.8)

X_train = X[:split_point]
X_test = X[split_point:]

y_train = y[:split_point]
y_test = y[split_point:]

# apply standart scalar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

SCALER_FILEPATH = os.path.join(MODEL_DIR, 'feature_scaler.pkl')
stars_utils.save_scaler(scaler, SCALER_FILEPATH)

# Fit and transform the training targets
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)

# Transform the testing targets
y_test_scaled = y_scaler.transform(y_test)

print(f"Total data points: {len(X)}")
print(f"Training period ends at index {split_point} (Date: {df.iloc[split_point]['Time_UTC']})")
print(f"Testing period starts at index {split_point} (Date: {df.iloc[split_point]['Time_UTC']})")

NameError: name 'mars_sf_df' is not defined

In [17]:
# MLP NN Implemantation

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# use early stopping for when there is no improvement for 150 epochs
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=150, 
    restore_best_weights=True
)

# Reduce the LR by 50% (factor=0.5), If no improvement for 50 epochs, drop the LR, Don't let the LR drop below 1e-7 (0.0000001)
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,       
    patience=50,      
    min_lr=1e-7       
)

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 
regularizer_strength = 0.001 

model = Sequential([
    # Apply kernel_regularizer to hidden layers
    Dense(128, activation='relu', kernel_regularizer=l2(0.0001), 
          input_shape=(X_train_scaled.shape[1],)), 
    
    Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.0001)), 
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),

    Dense(3, activation='linear') 
])

model.compile(optimizer=custom_optimizer, loss='mse') 

# use 10000 spochs with batch size of 32
print("\nStarting Neural Network training (100 epochs)...")

history = model.fit(
    X_train_scaled, 
    y_train_scaled, 
    epochs=10000,             
    batch_size=32,          
    validation_data=(X_test_scaled, y_test_scaled),
    callbacks=[early_stopping, lr_scheduler],
    verbose=1              
)

print("Model training complete.")


Starting Neural Network training (100 epochs)...
Epoch 1/10000


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step - loss: 0.0532 - val_loss: 0.0392 - learning_rate: 0.0010
Epoch 2/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 0.0209 - val_loss: 0.0257 - learning_rate: 0.0010
Epoch 3/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - loss: 0.0129 - val_loss: 0.0178 - learning_rate: 0.0010
Epoch 4/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 0.0084 - val_loss: 0.0131 - learning_rate: 0.0010
Epoch 5/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - loss: 0.0056 - val_loss: 0.0149 - learning_rate: 0.0010
Epoch 6/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - loss: 0.0043 - val_loss: 0.0077 - learning_rate: 0.0010
Epoch 7/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 0.0032 - val_loss: 0.0087 - learning_rate: 0.0010
Epoch 8/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - loss: 0.0028 - val_loss: 0.0044 - learning_rate: 0.0010
Epoch 9/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: 0.0022 - val_loss: 0.0041 - l

KeyboardInterrupt: 

In [12]:
# Predict on the test set
y_pred_scaled = model.predict(X_test_scaled)

# get inverse value (becuase of the scaling)
y_pred_mlp_au = y_scaler.inverse_transform(y_pred_scaled)

# Calculate the loss (MSE) on the test set
y_test_np = y_test.values
loss_mse = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
rmse_au = np.sqrt(loss_mse) 

# Calculate RMSE for each coordinate individually
rmse_x = np.sqrt(mean_squared_error(y_test_np[:, 0], y_pred_mlp_au[:, 0]))
rmse_y = np.sqrt(mean_squared_error(y_test_np[:, 1], y_pred_mlp_au[:, 1]))
rmse_z = np.sqrt(mean_squared_error(y_test_np[:, 2], y_pred_mlp_au[:, 2]))

print("\n--- Model Evaluation (Neural Network Test Set) ---")
print(f"Overall Averaged RMSE: {rmse_au:.6f} AU")
print(f"X-coordinate RMSE: {rmse_x:.6f} AU")
print(f"Y-coordinate RMSE: {rmse_y:.6f} AU")
print(f"Z-coordinate RMSE: {rmse_z:.6f} AU")

126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step

--- Model Evaluation (Neural Network Test Set) ---
Overall Averaged RMSE: 0.019528 AU
X-coordinate RMSE: 0.003944 AU
Y-coordinate RMSE: 0.003822 AU
Z-coordinate RMSE: 0.001565 AU


In [ ]:
# save the model
MODEL_DIR = 'models'
MODEL_FILENAME = 'mars_position_predictor_final.keras' 

# # Create the directory if it doesn't exist
# os.makedirs(MODEL_DIR, exist_ok=True) 

# Save the model
model.save(os.path.join(MODEL_DIR, MODEL_FILENAME))

print(f"Model saved successfully to {os.path.join(MODEL_DIR, MODEL_FILENAME)}")

Model saved successfully to models/mars_position_predictor_final.keras


In [11]:
# Load the model
model_path = os.path.join(MODEL_DIR, MODEL_FILENAME)
model = tf.keras.models.load_model(model_path)

In [ ]:
# Run the conversion function
radec_predictions = stars_utils.xyz_to_radec(y_pred_mlp_au)

print("--- Predicted RA/Dec Coordinates (Degrees) ---")
print(radec_predictions.head())

--- Predicted RA/Dec Coordinates (Degrees) ---
   Predicted_RA_deg  Predicted_Dec_deg
0        191.896774          -2.759812
1        192.311584          -2.923764
2        192.723251          -3.086056
3        193.131683          -3.246643
4        193.522125          -3.398860


In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# --- MODEL CONFIGURATION ---
NUM_MODELS = 3 # Number of MLP models in the ensemble
# Set a high number of epochs since Early Stopping will handle when to stop
EPOCHS = 5000   
INPUT_SHAPE = X_train_scaled.shape[1] # Number of input features
models = []
y_pred_list = []

# --- Define Callbacks for Training ---
early_stopping_callback = EarlyStopping(
    monitor='val_loss', # Monitors validation loss
    patience=150,        # Stops training after 150 epochs of no improvement
    restore_best_weights=True,
    min_delta = 1e-7 
    )

lr_on_plateau_callback = ReduceLROnPlateau(
    monitor='val_loss', # Monitors validation loss
    factor=0.5,         # Reduces learning rate by 50%
    patience=50,        # If no improvement for 50 epochs, reduce LR
    min_lr=1e-7         # The minimum learning rate to allow
)

callbacks = [early_stopping_callback, lr_on_plateau_callback]

# --- 1. BUILD AND TRAIN THE INDIVIDUAL MLP MODELS ---
print(f"Building and training {NUM_MODELS} diverse MLP models...")

for i in range(NUM_MODELS):
    # Use a different random seed for each model to ensure diverse weight initializations
    tf.keras.utils.set_random_seed(i + 1)
    
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.0001), 
              input_shape=(INPUT_SHAPE,)), 
        
        Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.0001)), 
        
        Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),

        Dense(3, activation='linear') 
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print(f"\n--- Training MLP Model {i+1} ---")
    model.fit(
        X_train_scaled, 
        y_train.values, 
        epochs=EPOCHS, 
        validation_data=(X_test_scaled, y_test.values),
        callbacks=callbacks, # Pass the list of callbacks here
        verbose=1 # Changed to 1 to show the callback messages
    )
    
    models.append(model)
    
    # Generate predictions for the current model and store them
    y_pred = model.predict(X_test_scaled)
    y_pred_list.append(y_pred)
    
    # Calculate and print the individual model's RMSE for comparison
    rmse_x = np.sqrt(mean_squared_error(y_test.values[:, 0], y_pred[:, 0]))
    rmse_y = np.sqrt(mean_squared_error(y_test.values[:, 1], y_pred[:, 1]))
    rmse_z = np.sqrt(mean_squared_error(y_test.values[:, 2], y_pred[:, 2]))
    overall_rmse = np.mean([rmse_x, rmse_y, rmse_z])
    
    print(f"Model {i+1} Test Set RMSE: {overall_rmse:.6f} AU")

# --- 2. CREATE THE ENSEMBLE PREDICTION ---
# The ensemble prediction is the simple average of all individual model predictions.
y_ensemble_pred_au = np.mean(y_pred_list, axis=0)

# --- 3. EVALUATE THE ENSEMBLE PERFORMANCE ---
print("\n--- Final Ensemble Evaluation ---")
rmse_x_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 0], y_ensemble_pred_au[:, 0]))
rmse_y_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 1], y_ensemble_pred_au[:, 1]))
rmse_z_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 2], y_ensemble_pred_au[:, 2]))
overall_ensemble_rmse = np.mean([rmse_x_ensemble, rmse_y_ensemble, rmse_z_ensemble])

print(f"Ensemble X-coordinate RMSE: {rmse_x_ensemble:.6f} AU")
print(f"Ensemble Y-coordinate RMSE: {rmse_y_ensemble:.6f} AU")
print(f"Ensemble Z-coordinate RMSE: {rmse_z_ensemble:.6f} AU")
print(f"Overall Ensemble RMSE: {overall_ensemble_rmse:.6f} AU")

# --- 4. Compare with your best single MLP result ---
# Replace the value below with your actual best MLP RMSE (approx. 0.0031)
BEST_SINGLE_MLP_RMSE = 0.003100
print("\n--- Final Comparison ---")
print(f"Best Single MLP Overall RMSE: {BEST_SINGLE_MLP_RMSE:.6f} AU")
print(f"Multi-MLP Ensemble Overall RMSE: {overall_ensemble_rmse:.6f} AU")

# Determine if the ensemble is better
if overall_ensemble_rmse < BEST_SINGLE_MLP_RMSE:
    print("\nSUCCESS: The multi-model MLP ensemble is more accurate!")
else:
    print("\nRESULT: The single MLP model remains the most accurate.")


Building and training 3 diverse MLP models...

--- Training MLP Model 1 ---
Epoch 1/5000


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 937us/step - loss: 0.0556 - val_loss: 0.0392 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 0.0226 - val_loss: 0.0258 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - loss: 0.0144 - val_loss: 0.0168 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - loss: 0.0093 - val_loss: 0.0109 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - loss: 0.0063 - val_loss: 0.0084 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: 0.0047 - val_loss: 0.0067 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 0.0036 - val_loss: 0.0057 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 0.0031 - val_loss: 0.0058 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: 0.0024 - val_loss: 0.0045 - learning_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 864us/step - loss: 0.0592 - val_loss: 0.0410 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - loss: 0.0218 - val_loss: 0.0235 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 0.0138 - val_loss: 0.0158 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - loss: 0.0092 - val_loss: 0.0117 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - loss: 0.0063 - val_loss: 0.0095 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - loss: 0.0046 - val_loss: 0.0091 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - loss: 0.0039 - val_loss: 0.0092 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - loss: 0.0031 - val_loss: 0.0077 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - loss: 0.0027 - val_loss: 0.0061 - learning_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 859us/step - loss: 0.0525 - val_loss: 0.0503 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.0224 - val_loss: 0.0318 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 0.0141 - val_loss: 0.0203 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - loss: 0.0091 - val_loss: 0.0187 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - loss: 0.0063 - val_loss: 0.0191 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - loss: 0.0047 - val_loss: 0.0075 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.0036 - val_loss: 0.0052 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - loss: 0.0028 - val_loss: 0.0087 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 0.0024 - val_loss: 0.0039 - learning_

In [32]:
MODEL_DIR = 'models'
MODEL_FILENAME = 'mars_position_predictor_mm1.keras' 

# # Create the directory if it doesn't exist
# os.makedirs(MODEL_DIR, exist_ok=True) 

# Save the model
models[0].save(os.path.join(MODEL_DIR, MODEL_FILENAME))

print(f"Model saved successfully to {os.path.join(MODEL_DIR, MODEL_FILENAME)}")

Model saved successfully to models/mars_position_predictor_mm1.keras


In [31]:
y_pred_mm1 = y_pred_list[0]
print("\n--- MM1 Ensemble Evaluation ---")
rmse_x_mm1 = np.sqrt(mean_squared_error(y_test.values[:, 0], y_pred_mm1[:, 0]))
rmse_y_mm1 = np.sqrt(mean_squared_error(y_test.values[:, 1], y_pred_mm1[:, 1]))
rmse_z_mm1 = np.sqrt(mean_squared_error(y_test.values[:, 2], y_pred_mm1[:, 2]))
overall_mm1_rmse = np.mean([rmse_x_mm1, rmse_y_mm1, rmse_z_mm1])

print(f"Ensemble X-coordinate RMSE: {rmse_x_mm1:.6f} AU")
print(f"Ensemble Y-coordinate RMSE: {rmse_y_mm1:.6f} AU")
print(f"Ensemble Z-coordinate RMSE: {rmse_z_mm1:.6f} AU")
print(f"Overall Ensemble RMSE: {overall_mm1_rmse:.6f} AU")


--- MM1 Ensemble Evaluation ---
Ensemble X-coordinate RMSE: 0.003033 AU
Ensemble Y-coordinate RMSE: 0.003884 AU
Ensemble Z-coordinate RMSE: 0.001660 AU
Overall Ensemble RMSE: 0.002859 AU


In [30]:
y_pred_mm1 = y_pred_list[0]
y_pred_list_ens = [y_pred_mlp_au, y_pred_mm1]
y_ensemble_pred_au = np.mean(y_pred_list_ens, axis=0)

# --- 3. EVALUATE THE ENSEMBLE PERFORMANCE ---
print("\n--- Final Ensemble Evaluation ---")
rmse_x_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 0], y_ensemble_pred_au[:, 0]))
rmse_y_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 1], y_ensemble_pred_au[:, 1]))
rmse_z_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 2], y_ensemble_pred_au[:, 2]))
overall_ensemble_rmse = np.mean([rmse_x_ensemble, rmse_y_ensemble, rmse_z_ensemble])

print(f"Ensemble X-coordinate RMSE: {rmse_x_ensemble:.6f} AU")
print(f"Ensemble Y-coordinate RMSE: {rmse_y_ensemble:.6f} AU")
print(f"Ensemble Z-coordinate RMSE: {rmse_z_ensemble:.6f} AU")
print(f"Overall Ensemble RMSE: {overall_ensemble_rmse:.6f} AU")

# --- 4. Compare with your best single MLP result ---
# Replace the value below with your actual best MLP RMSE (approx. 0.0031)
BEST_SINGLE_MLP_RMSE = 0.003100
print("\n--- Final Comparison ---")
print(f"Best Single MLP Overall RMSE: {BEST_SINGLE_MLP_RMSE:.6f} AU")
print(f"Multi-MLP Ensemble Overall RMSE: {overall_ensemble_rmse:.6f} AU")

# Determine if the ensemble is better
if overall_ensemble_rmse < BEST_SINGLE_MLP_RMSE:
    print("\nSUCCESS: The multi-model MLP ensemble is more accurate!")
else:
    print("\nRESULT: The single MLP model remains the most accurate.")


--- Final Ensemble Evaluation ---
Ensemble X-coordinate RMSE: 0.002818 AU
Ensemble Y-coordinate RMSE: 0.003294 AU
Ensemble Z-coordinate RMSE: 0.001294 AU
Overall Ensemble RMSE: 0.002468 AU

--- Final Comparison ---
Best Single MLP Overall RMSE: 0.003100 AU
Multi-MLP Ensemble Overall RMSE: 0.002468 AU

SUCCESS: The multi-model MLP ensemble is more accurate!


In [27]:

# --- CRITICAL FIX REMOVED: tf.config.run_functions_eagerly(True) has been removed.
# This should revert to fast Graph Execution mode.

# --- ASSUME YOUR DATA IS ALREADY LOADED AND PREPROCESSED ---
# X_train_scaled, y_train, X_test_scaled, y_test
# (Assuming your data variables are correctly loaded in the environment)

# --- NEW: OPTIMIZE DATA LOADING FOR PERFORMANCE (FASTEST PATH) ---
# We define raw slices once and optimize the pipeline per model inside the loop.
BATCH_SIZE = 32 # Standard batch size
AUTOTUNE = tf.data.AUTOTUNE 

# 1. Convert NumPy arrays to raw, unshuffled tf.data.Dataset slices (defined once)
raw_train_dataset = Dataset.from_tensor_slices((X_train_scaled, y_train.values.astype(np.float32)))
raw_test_dataset = Dataset.from_tensor_slices((X_test_scaled, y_test.values.astype(np.float32)))


# --- MODEL CONFIGURATION ---
NUM_MODELS = 9 # Assuming you set this to 9 in your environment
# Set a high number of epochs since Early Stopping will handle when to stop
EPOCHS = 5000   
INPUT_SHAPE = X_train_scaled.shape[1] # Number of input features
models = []
y_pred_list = []

# Variables for tracking the best individual model
best_individual_rmse = float('inf')
best_model_index = -1
best_model_seed = -1
best_model_predictions = None
# Replace the value below with your actual best single MLP RMSE (approx. 0.0031)
BEST_SINGLE_MLP_RMSE = 0.003100


# --- Define Callbacks for Training ---
early_stopping_callback = EarlyStopping(
    monitor='val_loss', 
    patience=150,        
    restore_best_weights=True,
    # REVERTED: Removed min_delta to use the default (0.0), which aligns with the original, best-performing setup.
)

lr_on_plateau_callback = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,         
    patience=50,        
    min_lr=1e-7         
)

callbacks = [early_stopping_callback, lr_on_plateau_callback]

# --- 1. BUILD AND TRAIN THE INDIVIDUAL MLP MODELS ---
print(f"Building and training {NUM_MODELS} diverse MLP models...")

for i in range(NUM_MODELS):
    tf.keras.backend.clear_session()
    current_seed = i + 1
    # Set the seed for model weight initialization
    tf.keras.utils.set_random_seed(current_seed)
    
    # --- CRITICAL FIX: Create unique shuffled datasets for each model ---
    # 2. Shuffle, batch, and prefetch for optimal pipeline speed
    # We use the current_seed to ensure each model sees the data in a different order.
    train_dataset = raw_train_dataset.shuffle(buffer_size=1024, seed=current_seed).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    test_dataset = raw_test_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    
    # --- MODEL ARCHITECTURE REVERTED TO ORIGINAL HIGH-PERFORMANCE STRUCTURE ---
    model = Sequential([
        Dense(128, activation='relu', 
              kernel_regularizer=l2(0.0001), # Reverted L2 to original 0.0001
              input_shape=(INPUT_SHAPE,)), 
        
        Dense(256, activation='relu',
              kernel_regularizer=l2(0.0001)), # Reverted L2 to original 0.0001
        
        Dense(128, activation='relu', 
              kernel_regularizer=l2(0.0001)), # Reverted L2 to original 0.0001
        
        Dense(64, activation='relu'),

        Dense(3, activation='linear') 
    ])
    # --- END REVERT ---
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error')
    
    print(f"\n--- Training MLP Model {i+1} (Seed: {current_seed}) ---")
    
    # Use the optimized datasets for fastest training
    model.fit(
        train_dataset, 
        epochs=EPOCHS, 
        validation_data=test_dataset, 
        callbacks=callbacks, 
        verbose=1 
    )
    
    models.append(model)
    
    # Generate predictions using the raw NumPy array (this usually runs fine)
    y_pred = model.predict(X_test_scaled)
    y_pred_list.append(y_pred)
    
    # Calculate and print the individual model's RMSE for comparison
    rmse_x = np.sqrt(mean_squared_error(y_test.values[:, 0], y_pred[:, 0]))
    rmse_y = np.sqrt(mean_squared_error(y_test.values[:, 1], y_pred[:, 1]))
    rmse_z = np.sqrt(mean_squared_error(y_test.values[:, 2], y_pred[:, 2]))
    overall_rmse = np.mean([rmse_x, rmse_y, rmse_z])
    
    print(f"Model {i+1} Test Set RMSE: {overall_rmse:.6f} AU")
    
    if overall_rmse < best_individual_rmse:
        best_individual_rmse = overall_rmse
        best_model_index = i
        best_model_seed = current_seed
        best_model_predictions = y_pred.copy()

# --- 2. CREATE THE ENSEMBLE PREDICTION ---
y_ensemble_pred_au = np.mean(y_pred_list, axis=0)

# --- 3. EVALUATE THE ENSEMBLE PERFORMANCE ---
print("\n--- Final Ensemble Evaluation ---")
rmse_x_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 0], y_ensemble_pred_au[:, 0]))
rmse_y_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 1], y_ensemble_pred_au[:, 1]))
rmse_z_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 2], y_ensemble_pred_au[:, 2]))
overall_ensemble_rmse = np.mean([rmse_x_ensemble, rmse_y_ensemble, rmse_z_ensemble])

print(f"Ensemble X-coordinate RMSE: {rmse_x_ensemble:.6f} AU")
print(f"Ensemble Y-coordinate RMSE: {rmse_y_ensemble:.6f} AU")
print(f"Ensemble Z-coordinate RMSE: {overall_ensemble_rmse:.6f} AU")

# --- 4. Final Comparison and Extraction of Best Model ---
print("\n--- Final Comparison ---")
print(f"Best Single MLP Overall RMSE: {BEST_SINGLE_MLP_RMSE:.6f} AU")
print(f"Multi-MLP Ensemble Overall RMSE: {overall_ensemble_rmse:.6f} AU")

print(f"\n--- Best Individual Model Summary ---")
print(f"Best Individual Model (Index {best_model_index+1}) achieved RMSE: {best_individual_rmse:.6f} AU")
print(f"The random seed used for this best model was: {best_model_seed}")

# --- 5. SAVING THE BEST MODEL ---
if overall_ensemble_rmse < best_individual_rmse and overall_ensemble_rmse < BEST_SINGLE_MLP_RMSE:
    print("\nCONCLUSION: The Multi-MLP Ensemble is the most accurate predictor.")
    MODEL_SAVE_NAME = "final_best_ensemble_predictions.npy"
    np.save(MODEL_SAVE_NAME, y_ensemble_pred_au)
    print(f"Saved FINAL ENSEMBLE PREDICTIONS to: {MODEL_SAVE_NAME}")
    
elif best_individual_rmse < overall_ensemble_rmse and best_individual_rmse < BEST_SINGLE_MLP_RMSE:
    print(f"\nCONCLUSION: A new individual MLP model (Index {best_model_index+1}) is the most accurate predictor.")
    
    final_best_model = models[best_model_index]
    MODEL_SAVE_NAME = f"best_mlp_model_seed_{best_model_seed}.h5" 
    
    # Check if file exists before saving
    if os.path.exists(MODEL_SAVE_NAME):
        print(f"Warning: File {MODEL_SAVE_NAME} already exists. Skipping save to prevent overwrite.")
    else:
        final_best_model.save(MODEL_SAVE_NAME)
        print(f"Saved FINAL BEST INDIVIDUAL MODEL to: {MODEL_SAVE_NAME}")
    
else:
    print("\nCONCLUSION: The original single MLP model remains the most accurate.")

Building and training 9 diverse MLP models...

--- Training MLP Model 1 (Seed: 1) ---
Epoch 1/5000


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 876us/step - loss: 0.0412 - val_loss: 0.0485 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - loss: 0.0168 - val_loss: 0.0272 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - loss: 0.0113 - val_loss: 0.0230 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - loss: 0.0078 - val_loss: 0.0124 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - loss: 0.0054 - val_loss: 0.0110 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.0042 - val_loss: 0.0083 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - loss: 0.0032 - val_loss: 0.0082 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 0.0024 - val_loss: 0.0048 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 0.0020 - val_loss: 0.0036 - learning_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - loss: 0.0476 - val_loss: 0.0428 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - loss: 0.0163 - val_loss: 0.0283 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.0111 - val_loss: 0.0217 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 0.0079 - val_loss: 0.0143 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - loss: 0.0056 - val_loss: 0.0135 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.0042 - val_loss: 0.0082 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.0032 - val_loss: 0.0061 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - loss: 0.0024 - val_loss: 0.0068 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - loss: 0.0023 - val_loss: 0.0055 - learning_

...

503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0480 - val_loss: 0.0334 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - loss: 0.0184 - val_loss: 0.0212 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - loss: 0.0123 - val_loss: 0.0164 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - loss: 0.0084 - val_loss: 0.0127 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0058 - val_loss: 0.0082 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - loss: 0.0043 - val_loss: 0.0092 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - loss: 0.0031 - val_loss: 0.0102 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0025 - val_loss: 0.0039 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - loss: 0.0022 - val_loss: 0.0037 - learning_rate: 

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - loss: 0.0441 - val_loss: 0.0416 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - loss: 0.0163 - val_loss: 0.0289 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 0.0111 - val_loss: 0.0339 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: 0.0076 - val_loss: 0.0201 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - loss: 0.0056 - val_loss: 0.0172 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - loss: 0.0040 - val_loss: 0.0086 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - loss: 0.0032 - val_loss: 0.0081 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - loss: 0.0027 - val_loss: 0.0127 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step - loss: 0.0021 - val_loss: 0.0051 - learning_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 812us/step - loss: 0.0428 - val_loss: 0.0382 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 0.0176 - val_loss: 0.0275 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - loss: 0.0121 - val_loss: 0.0216 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 0.0087 - val_loss: 0.0199 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - loss: 0.0061 - val_loss: 0.0143 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - loss: 0.0044 - val_loss: 0.0104 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 0.0036 - val_loss: 0.0103 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 0.0026 - val_loss: 0.0078 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - loss: 0.0022 - val_loss: 0.0093 - learning_

In [ ]:
MODEL_DIR = 'models'
MODELS_LIS = ['mm0','mm1','mm2']

models = stars_utils.models_loader(MODEL_DIR,MODELS_LIS)